In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [3]:
num_scen = 101 # set it to 1+whatever

In [4]:
# Initialize the DataFrame to store the combined data
combined_data_final = pd.DataFrame()

# Loop through each scenario
for k in range(1,num_scen):
    # Initialize DataFrame to store data for the current scenario
    combined_data = pd.DataFrame()
    
    # Read timestamps data
    sf = pd.read_csv(f"C:\\study material\\CS F266\\Hanoi_CMH\\Scenario-{k}\\Timestamps.csv")
    combined_data['Timestamp'] = sf['Timestamp'].tolist()
    
    # Read demands data for nodes
    for i in range(1, 33):
        df1 = pd.read_csv(f"C:\\study material\\CS F266\\Hanoi_CMH\\Scenario-{k}\\Demands\\Node_{i}.csv")
        combined_data[f"D_node-{i}"] = df1['Value'].tolist()
    
    # Read pressures data for nodes
    for y in range(1, 33):
        df2 = pd.read_csv(f"C:\\study material\\CS F266\\Hanoi_CMH\\Scenario-{k}\\Pressures\\Node_{y}.csv")
        combined_data[f"P_node-{y}"] = df2['Value'].tolist()
    
    # Read flows data for links
    for z in range(1, 35):
        df3 = pd.read_csv(f"C:\\study material\\CS F266\\Hanoi_CMH\\Scenario-{k}\\Flows\\Link_{z}.csv")
        combined_data[f"F_node-{z}"] = df3['Value'].tolist()
    
    # Concatenate data for the current scenario with the final combined data
    combined_data_final = pd.concat([combined_data_final, combined_data], ignore_index=True)

In [5]:
combined_data_final.shape

(1752000, 99)

In [6]:
new_columns = pd.DataFrame(0.0, index=combined_data_final.index, columns=[f"Leak_node-{i}" for i in range(1, 33)])

# Concatenate new_columns DataFrame to the original dataset
combined_data_final = pd.concat([combined_data_final, new_columns], axis=1)

In [7]:
combined_data_final.shape

(1752000, 131)

In [8]:
combined_data_final

,Timestamp,D_node-1,D_node-2,D_node-3,D_node-4,D_node-5,D_node-6,D_node-7,D_node-8,D_node-9,...,Leak_node-23,Leak_node-24,Leak_node-25,Leak_node-26,Leak_node-27,Leak_node-28,Leak_node-29,Leak_node-30,Leak_node-31,Leak_node-32
0,2017-01-01 00:00:00,-3337.2,151.2,140.4,21.6,104.4,169.2,219.6,86.4,90.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-01-01 00:30:00,-2973.6,136.8,122.4,18.0,100.8,154.8,230.4,79.2,72.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-01-01 01:00:00,-2584.8,118.8,100.8,18.0,82.8,140.4,194.4,64.8,72.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-01-01 01:30:00,-2419.2,108.0,93.6,14.4,75.6,144.0,176.4,57.6,61.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-01-01 02:00:00,-2196.0,100.8,82.8,14.4,72.0,133.2,147.6,54.0,61.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751995,2017-12-31 21:30:00,-5533.2,273.6,252.0,32.4,216.0,259.2,396.0,151.2,140.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1751996,2017-12-31 22:00:00,-5317.2,284.4,212.4,28.8,201.6,244.8,396.0,151.2,144.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1751997,2017-12-31 22:30:00,-4888.8,244.8,223.2,28.8,183.6,223.2,345.6,144.0,129.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1751998,2017-12-31 23:00:00,-4482.0,226.8,180.0,28.8,165.6,205.2,316.8,129.6,122.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# Iterate over scenarios
for k in range(1, num_scen):
    # Iterate over range(1, 35)
    for x in range(1, 35):
        file_path = f"C:\\study material\\CS F266\\Hanoi_CMH\\Scenario-{k}\\Leaks\\Leak_{x}_demand.csv"
        if os.path.exists(file_path):
            df4 = pd.read_csv(file_path)
            if not df4.empty:
                start_index = 17520*(k-1)
                end_index = 17520*k
                combined_data_final.loc[start_index:end_index-1, f'Leak_node-{x}'] = df4['Value'].values
                print("Values replaced successfully.")
            else:
                print(f"No data found in df4 for Leak_node-{x} in scenario {k}")

print("Replacement process completed.")

Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values replaced successfully.
Values rep

In [10]:
# checking
non_zero_values = combined_data_final[combined_data_final['Leak_node-6'] != 0]['Leak_node-6']
print(non_zero_values)


32944       46.8
32949       46.8
32950       46.8
32951       43.2
32952       46.8
           ...  
1149048    111.6
1149049    111.6
1149061    111.6
1149062    111.6
1149063    111.6
Name: Leak_node-6, Length: 17673, dtype: float64


In [11]:
leak_column_sums = combined_data_final.filter(like='Leak_node-').sum()
print("Sum of each leak column:")
print(leak_column_sums)

Sum of each leak column:
Leak_node-1            0.0
Leak_node-2      3889429.2
Leak_node-3     22812339.6
Leak_node-4     15230610.0
Leak_node-5      1304542.8
Leak_node-6     29147806.8
Leak_node-7     19842220.8
Leak_node-8      9609055.2
Leak_node-9      5215971.6
Leak_node-10     1459213.2
Leak_node-11     2007421.2
Leak_node-12     5282398.8
Leak_node-13    11498486.4
Leak_node-14     1641567.6
Leak_node-15     4228002.0
Leak_node-16     8674714.8
Leak_node-17     1540357.2
Leak_node-18     4142239.2
Leak_node-19     5046256.8
Leak_node-20    30660926.4
Leak_node-21    18301741.2
Leak_node-22     2016637.2
Leak_node-23     7901996.4
Leak_node-24           0.0
Leak_node-25         194.4
Leak_node-26    15732532.8
Leak_node-27       24548.4
Leak_node-28     1019710.8
Leak_node-29     5157136.8
Leak_node-30    11040296.4
Leak_node-31      577782.0
Leak_node-32    30791815.2
dtype: float64


In [12]:
combined_data_final.head()

,Timestamp,D_node-1,D_node-2,D_node-3,D_node-4,D_node-5,D_node-6,D_node-7,D_node-8,D_node-9,...,Leak_node-23,Leak_node-24,Leak_node-25,Leak_node-26,Leak_node-27,Leak_node-28,Leak_node-29,Leak_node-30,Leak_node-31,Leak_node-32
0,2017-01-01 00:00:00,-3337.2,151.2,140.4,21.6,104.4,169.2,219.6,86.4,90.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-01-01 00:30:00,-2973.6,136.8,122.4,18.0,100.8,154.8,230.4,79.2,72.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-01-01 01:00:00,-2584.8,118.8,100.8,18.0,82.8,140.4,194.4,64.8,72.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-01-01 01:30:00,-2419.2,108.0,93.6,14.4,75.6,144.0,176.4,57.6,61.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-01-01 02:00:00,-2196.0,100.8,82.8,14.4,72.0,133.2,147.6,54.0,61.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
combined_data_final['Leak_node-6'][33340]

46.8

In [14]:
# Assuming combined_data is already created
for column in combined_data_final.columns:
    if column.startswith('Leak_node-'):
        combined_data_final[column] = combined_data_final[column].apply(lambda x: 1 if x != 0.0 else 0)


In [15]:
combined_data_final['Leak_node-6'][33340]

1

In [16]:
combined_data_final['Leak_node-6'].sum()

17673

In [17]:
output_file_path = "combined_data_final_for_model_100scen.csv"
combined_data_final.to_csv(output_file_path, index=False)
print(f"Data saved to {output_file_path}")

Data saved to combined_data_final_for_model_100scen.csv
